In [53]:
import numpy as np
import pandas as pd

input_dir = 'data/'
train = pd.read_csv(input_dir + 'train.csv',encoding='utf_8')
test = pd.read_csv(input_dir + 'test.csv',encoding='utf_8')
resources = pd.read_csv(input_dir + 'resources.csv',encoding='utf_8')
submission = pd.read_csv(input_dir + 'sample_submission.csv')

resources['total_price'] = resources.quantity * resources.price
resources.head()
train.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [57]:
mean_total_price = pd.DataFrame(resources.groupby('id').total_price.mean()) 
mean_total_price.head()

,total_price
id,
p000001,208.407500
p000002,45.020000
p000003,74.742500
p000004,11.854947
p000005,175.577500


In [58]:
#Add id as column for merging
mean_total_price['id'] = mean_total_price.index

cols_to_drop = [
    'teacher_id',
    'project_title', 
    'project_essay_1', 
    'project_essay_2', 
    'project_essay_3', 
    'project_essay_4', 
    'project_subject_categories',
    'project_subject_subcategories',
    'project_grade_category',
    'project_resource_summary'
]

train = train.drop(cols_to_drop, axis=1, errors='ignore')

train = pd.merge(train, mean_total_price, on='id')
X_train = train.values.reshape(-1, 1) #reshape because of one column
#X_train=train
y_train = train.project_is_approved

In [59]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
n_folds = 10
kf = KFold(n_splits=n_folds)
Average_AUC = 0
for (train_index, test_index) in kf.split(X_train):
    X, X_val = X_train[train_index], X_train[test_index]
    y, y_val = y_train[train_index], y_train[test_index]
    clf = LogisticRegression()
    clf.fit(X, y)
    pred = clf.predict_proba(X_val)[:,1]
    AUC = roc_auc_score(y_val, pred)
    Average_AUC += AUC
    print("AUC: {}".format(AUC))
Average_AUC = Average_AUC / n_folds
print("Average_AUC: {}".format(Average_AUC))

ValueError: invalid literal for float(): 2016-05-15 20:25:14

In [ ]:
test = pd.merge(test, mean_total_price, on='id')

In [ ]:
test = test.drop(cols_to_drop, axis=1, errors='ignore')
X_test = test.values.reshape(-1, 1)
pred_test = clf.predict_proba(X_test)[:,1]

In [ ]:
submission.project_is_approved = pred_test
submission.head(5)

In [ ]:
submission.to_csv('submission.csv', index=False)